## Pure columnar format

In [340]:
#
#first_name:1,last_name:2,department:3,salary:4
#

import os

if os.path.exists('employees.col1'):
    os.remove('employees.col1')

with open('employees.col1', 'w') as f:
    f.write('{"columns": {"first_name":{"start": 0, "end": 0},"last_name":{"start": 0, "end": 0},"department":{"start": 0, "end": 0},"salary":{"start": 0, "end": 0}}}\n')
    

In [189]:
from faker import Faker
faker = Faker('en')

import numpy as np


In [190]:
def create_employee(salary):
    first_name = faker.first_name()
    last_name = faker.last_name()
    department = faker.random_element(elements=['Embedded','BigData','HR'])
    return {'first_name': first_name, 'last_name': last_name, 'department': department, 'salary': salary}

In [270]:

salaries = np.random.normal(16000, 6000, 10)

employees = []

for salary in salaries:
    employees.append(create_employee(int(salary)))

In [356]:
import json

def decode_file(path):
    with open(path, 'r') as f:
        header = json.loads(f.readline())
        end_of_meta = f.tell()
        columns = []
        for column_name in header['columns'].keys():
            column = header['columns'][column_name]
            f.seek(end_of_meta + column['start'])
            data_string = f.read(column['end'] - column['start']) 
            data = data_string.split(',') if data_string else []
            columnData = { 'name': column_name, 'data': data}
            columns.append(columnData)
    return (header, columns)


def encode_file(path, columns):
    new_header = {'columns': {}}
    current_offset = 0
    for column in columns:
        joined_data_len = len(','.join(column['data']))
        new_header['columns'][column['name']] = { 
            'start': current_offset, 
            'end': current_offset + joined_data_len
        }
        current_offset = current_offset + joined_data_len

    with open(path, 'w') as f:
            f.write(json.dumps(new_header) + '\n')
            for column in columns:
                f.write(','.join(column['data']))

    

In [290]:
def find_all(path):
    header, columns = decode_file(path)

    records = None
    for column in columns:
        for i, el in enumerate(column['data']):
            if not records:
                records = [None] * len(column['data'])
            if not records[i]:
                records[i] = {}
            records[i][column['name']] = el
        
        
    return records

In [357]:
def insert(path, employee):
    header, columns = decode_file(path)
    for column in columns:
        print(column['data'], column["name"])
        column['data'].append(f'{employee[column["name"]]}')
    encode_file(path, columns)
        

In [358]:
for e in employees:
    insert('employees.col1', e)

[] first_name
[] last_name
[] department
[] salary
['James'] first_name
['Meyer'] last_name
['BigData'] department
['15813'] salary
['James', 'Molly'] first_name
['Meyer', 'Pacheco'] last_name
['BigData', 'HR'] department
['15813', '11311'] salary
['James', 'Molly', 'Amy'] first_name
['Meyer', 'Pacheco', 'Moore'] last_name
['BigData', 'HR', 'Embedded'] department
['15813', '11311', '16454'] salary
['James', 'Molly', 'Amy', 'Sandra'] first_name
['Meyer', 'Pacheco', 'Moore', 'Baker'] last_name
['BigData', 'HR', 'Embedded', 'HR'] department
['15813', '11311', '16454', '14939'] salary
['James', 'Molly', 'Amy', 'Sandra', 'Kathleen'] first_name
['Meyer', 'Pacheco', 'Moore', 'Baker', 'Peterson'] last_name
['BigData', 'HR', 'Embedded', 'HR', 'HR'] department
['15813', '11311', '16454', '14939', '11764'] salary
['James', 'Molly', 'Amy', 'Sandra', 'Kathleen', 'Rhonda'] first_name
['Meyer', 'Pacheco', 'Moore', 'Baker', 'Peterson', 'Branch'] last_name
['BigData', 'HR', 'Embedded', 'HR', 'HR', 'Emb

In [247]:
def delete_eq(path, column_name, value):    
    header, columns = decode_file(path)
    
    searched_column = next(col for col in columns if col['name'] == column_name)
    found_indices = [i for i, r in enumerate(searched_column['data']) if r == value]

    for column in columns:
        for index in found_indices:
            del column['data'][index]
            
    print(columns)
    encode_file(path, columns)


In [225]:
def update_eq(path, searched_column_name, searched_value, new_record):
    header, columns = decode_file(path)

    searched_column = next(col for col in columns if col['name'] == searched_column_name)
    found_index = searched_column['data'].index(searched_value)

    for column in columns:
        column[index] = new_record[column['name']]
    
    encode_file(path, columns)

    

In [315]:
def find_eq_full_search(path, column_name, value, column_names):
    header, columns = decode_file(path)
    
    searched_column = next(col for col in columns if col['name'] == column_name)
    found_indices = [i for i, r in enumerate(searched_column['data']) if r == value]
  
    
    records = []
    for column in columns:
        for i, found_index in enumerate(found_indices):
            if len(records) <= i:
                records.append({})            
            if not column_names or len(column_names) == 0 or column['name'] in column_names:
                records[i][column['name']] = column['data'][found_index]
        
    return records


In [345]:
def find_eq(path, column_name, value, select_columns): # column_names => selected_columns
    with open(path, 'r') as f:
        meta = json.loads(f.readline())
        ref_offset = f.tell() # ref offset
        
        filter_column = meta['columns'][column_name]
    
        
        # znajdz indeksy na których pojawia sie dana wartosc
        
        start, end = filter_column['start'], filter_column['end']
        f.seek(ref_offset + start)
        raw_values = f.read(end - start) 
        values = raw_values and raw_values.split(',') or []
        found_indices = [i for i, r in enumerate(values) if r == value]
                
        data = []
        
        # excercise: majac dane liste indeksów - wyciagnij całe rekordy
        
        
        # excercise: majac dane liste indeksów - wyciagnij liste kolumn
        
        for column in select_columns:
            start, end = meta['columns'][column]['start'], meta['columns'][column]['end']
            
            f.seek(ref_offset + start)
            raw_values = f.read(end - start) 
            values = raw_values and raw_values.split(',') or []
            
            data.append([values[index] for index in found_indices])
        
    display(select_columns, list(zip(*data)))

In [362]:
find_eq('employees.col1', 'department', 'BigData', ['first_name', 'last_name', 'department'])

+------------+------------+------------+
| first_name | last_name  | department |
+============+============+============+
| James      | Meyer      | BigData    |
+------------+------------+------------+
| Katherine  | Strickland | BigData    |
+------------+------------+------------+
| Michael    | Santiago   | BigData    |
+------------+------------+------------+
| Richard    | Dawson     | BigData    |
+------------+------------+------------+


In [317]:
def find_gt(path, column_name, value, column_names):
    header, columns = decode_file(path)
    
    with open(path, 'r') as f:
        header = json.loads(f.readline())
        end_of_meta = f.tell()
        
        filter_column = [col for col in header['columns'] if col['name'] == column_name][0]
        
        f.seek(end_of_meta + filter_column['start_offset'])
        data_string = f.read(filter_column['end_offset'] - filter_column['start_offset']) 
        data = data_string.split(',') if data_string else []
        searched_column = { 'name': filter_column['name'], 'data': data}
        found_indices = [i for i, r in enumerate(searched_column['data']) if r > value]
        
        selected_columns = [col for col in header['columns'] if not column_names or len(column_names) == 0 or col['name'] in column_names]
        columns = []
        for column in selected_columns:
            f.seek(end_of_meta + column['start_offset'])
            data_string = f.read(column['end_offset'] - column['start_offset']) 
            data = data_string.split(',') if data_string else []
            columnData = { 'name': column['name'], 'data': data}
            columns.append(columnData)
        
    
    records = []
    for column in columns:
        for i, found_index in enumerate(found_indices):
            if len(records) <= i:
                records.append({})            
            records[i][column['name']] = column['data'][found_index]
        
        
    return records

In [319]:
def find_lt(path, column_name, value, column_names):
    header, columns = decode_file(path)
    
    with open(path, 'r') as f:
        header = json.loads(f.readline())
        end_of_meta = f.tell()
        
        filter_column = [col for col in header['columns'] if col['name'] == column_name][0]
        
        f.seek(end_of_meta + filter_column['start_offset'])
        data_string = f.read(filter_column['end_offset'] - filter_column['start_offset']) 
        data = data_string.split(',') if data_string else []
        searched_column = { 'name': filter_column['name'], 'data': data}
        found_indices = [i for i, r in enumerate(searched_column['data']) if r < value]
        
        selected_columns = [col for col in header['columns'] if not column_names or len(column_names) == 0 or col['name'] in column_names]
        columns = []
        for column in selected_columns:
            f.seek(end_of_meta + column['start_offset'])
            data_string = f.read(column['end_offset'] - column['start_offset']) 
            data = data_string.split(',') if data_string else []
            columnData = { 'name': column['name'], 'data': data}
            columns.append(columnData)
        
    
    records = []
    for column in columns:
        for i, found_index in enumerate(found_indices):
            if len(records) <= i:
                records.append({})            
            records[i][column['name']] = column['data'][found_index]
        
        
    return records

In [328]:
path = 'employees.col1'

find_eq(path, 'first_name', 'James', [])
# find_all(path)

[{'first_name': 'James',
  'last_name': 'Meyer',
  'department': 'BigData',
  'salary': '15813'}]

In [359]:
!cat employees.col1

{"columns": {"first_name": {"start": 0, "end": 68}, "last_name": {"start": 68, "end": 145}, "department": {"start": 145, "end": 206}, "salary": {"start": 206, "end": 264}}}
James,Molly,Amy,Sandra,Kathleen,Rhonda,Katherine,Michael,Richard,AmyMeyer,Pacheco,Moore,Baker,Peterson,Branch,Strickland,Santiago,Dawson,MartinezBigData,HR,Embedded,HR,HR,Embedded,BigData,BigData,BigData,HR15813,11311,16454,14939,11764,11218,27736,19568,4625,16704

1. Format - przedstaw plik
    a. ćwiczenie - funkcja zwracająca listę wartości danej kolumny (meta, f.seek(start), f.read(end-start))
2. 

In [339]:


from texttable import Texttable


def display(headers, rows):
    table = Texttable()
    table.add_rows([headers] + rows)
    print(table.draw())
    


selected_names = ['name', 'age']
values = [
    ['Maciek', 'Bartek'],
    [33, 28],
]

# print(list(zip(values)))

display(selected_names, list(zip(*values)))

+--------+-----+
|  name  | age |
+========+=====+
| Maciek | 33  |
+--------+-----+
| Bartek | 28  |
+--------+-----+
